In [1]:
import pandas as pd
import numpy as np

In [2]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
import bokeh.palettes as bpal
from bokeh.models import CategoricalColorMapper
from bokeh.transform import factor_cmap, linear_cmap
from bokeh.models import Legend, LinearAxis, Range1d, DatetimeTickFormatter
from bokeh.io import export_png
output_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

Loading BokehJS ...

In [3]:
from glob import glob
import json
from pprint import pprint

<font color = 'purple'><font size = 4>
    Am going to preprocess the various datetime-related features, and temperature features, creating feature dataframes for each NEM region/state

In [4]:
temp_options = glob("../../BoM/Temp-csvs/processed-temps/*.csv")
temp_options

['../../BoM/Temp-csvs/processed-temps/20190223_26_temps_processed.csv',
 '../../BoM/Temp-csvs/processed-temps/20190222_26_temps_processed.csv',
 '../../BoM/Temp-csvs/processed-temps/20190219_26_temps_processed.csv']

In [5]:
temp_fpath = temp_options[0]
temp_fpath

'../../BoM/Temp-csvs/processed-temps/20190223_26_temps_processed.csv'

In [6]:
options_demand = glob("../data/ready-for-model/*demand*.csv")
options_demand

['../data/ready-for-model/2009-18_NEMtotaldemand.csv']

In [7]:
file_demand = options_demand[0]
file_demand

'../data/ready-for-model/2009-18_NEMtotaldemand.csv'

In [8]:
find_tempdata = glob("../../BoM/Temp-csvs/*.json")
find_tempdata

['../../BoM/Temp-csvs/station-data.json',
 '../../BoM/Temp-csvs/merged-data.json']

In [9]:
jstempdict = find_tempdata[0]
jstempdict

'../../BoM/Temp-csvs/station-data.json'

In [10]:
with open(find_tempdata[1]) as infile:
    checkmerged = json.load(infile)

In [11]:
bomstatus = pd.DataFrame(checkmerged).T
bomstatus.head(2)

,MaxT,MinT,Name,latest,oldest
61078,yes,yes,WILLIAMTOWN-RAAF,2019-02-19,2009-01-01
39083,yes,yes,ROCKHAMPTON,2019-02-07,2009-01-01


In [12]:
bomstatus[(bomstatus.oldest > '2009-01-01') | (bomstatus.MaxT != 'yes') | (bomstatus.MinT != 'yes')]
## double-checking that none are missing years of data - was an issue with some stations, had to get different ones

,MaxT,MinT,Name,latest,oldest


In [13]:
hol_options = glob('../../Dates/*.csv')
hol_options

['../../Dates/australianpublicholidays-201516.csv',
 '../../Dates/australianpublicholidays-201617.csv',
 '../../Dates/australianpublicholidays-201415.csv',
 '../../Dates/20190221_publicholiday_DT-series_2009-18.csv',
 '../../Dates/allpubhols.csv',
 '../../Dates/australian_public_holidays_2019.csv',
 '../../Dates/australianpublicholidays-201718.csv']

In [14]:
holfile = hol_options[3]
holfile

'../../Dates/20190221_publicholiday_DT-series_2009-18.csv'

In [15]:
dfdemand = pd.read_csv(file_demand, index_col=0, parse_dates=[0])
dfdemand.head()

,NSW1,QLD1,SA1,TAS1,VIC1,NEMtotal
SETTLEMENTDATE,,,,,,
2009-01-01 00:30:00,7535.00,5611.54,1310.89,909.71,4799.87,20167.01
2009-01-01 01:00:00,7229.24,5457.34,1272.69,896.63,4646.21,19502.11
2009-01-01 01:30:00,6857.62,5294.12,1178.87,897.52,4950.16,19178.29
2009-01-01 02:00:00,6535.05,5153.47,1130.78,906.22,4755.46,18480.98
2009-01-01 02:30:00,6287.88,5060.33,1059.53,893.19,4545.67,17846.60


In [16]:
dfdemand['Date'] = pd.to_datetime(dfdemand.index.date)
dfdemand.head(2)

,NSW1,QLD1,SA1,TAS1,VIC1,NEMtotal,Date
SETTLEMENTDATE,,,,,,,
2009-01-01 00:30:00,7535.00,5611.54,1310.89,909.71,4799.87,20167.01,2009-01-01
2009-01-01 01:00:00,7229.24,5457.34,1272.69,896.63,4646.21,19502.11,2009-01-01


In [17]:
redemand_cols = {k:v for k,v in zip(dfdemand.columns[:-2], [d.split('1')[0] for d in dfdemand.columns[:-2]])}
print(redemand_cols)
redemand_cols['NEMtotal'] = 'NEM'
print(redemand_cols)

{'NSW1': 'NSW', 'QLD1': 'QLD', 'SA1': 'SA', 'TAS1': 'TAS', 'VIC1': 'VIC'}
{'NSW1': 'NSW', 'QLD1': 'QLD', 'SA1': 'SA', 'TAS1': 'TAS', 'VIC1': 'VIC', 'NEMtotal': 'NEM'}


In [18]:
dfdemand.rename(columns=redemand_cols, inplace=True)
dfdemand.head(2)

,NSW,QLD,SA,TAS,VIC,NEM,Date
SETTLEMENTDATE,,,,,,,
2009-01-01 00:30:00,7535.00,5611.54,1310.89,909.71,4799.87,20167.01,2009-01-01
2009-01-01 01:00:00,7229.24,5457.34,1272.69,896.63,4646.21,19502.11,2009-01-01


In [19]:
dftemps = pd.read_csv(temp_fpath, index_col=0, parse_dates=[0])
dftemps.head(2)

,Year,Month,WILLIAMTOWN-RAAF_MaxT_61078,ROCKHAMPTON_MaxT_39083,ARMIDALE_MinT_56037,MILDURA-AIRPORT_MinT_76031,CAPE-NELSON_MaxT_90184,ALBION-PARK_MaxT_68241,BUNDABERG-AERO_MaxT_39128,BROKEN-HILL-AIRPORT_MinT_47048,...,MURRAY-BRIDGE_MaxT_24584,CANBERRA_MinT_70351,HOBART_MaxT_94029,ALBURY-AIRPORT_MinT_72160,SYDNEY-AIRPORT-AMO_MinT_66037,LAUNCESTON_MinT_91237,GLADSTONE-AIRPORT_MinT_39326,GOLD-COAST-SEAWAY_MaxT_40764,PORT-AUGUSTA_MinT_18201,DUBBO_MinT_65070
2009-01-01,2009,1,34.0,37.3,6.5,13.8,15.9,34.0,34.0,18.3,...,23.8,11.1,17.5,11.3,17.7,12.6,26.3,31.0,17.5,10.8
2009-01-02,2009,1,24.5,27.1,15.4,8.9,16.0,21.0,28.4,11.7,...,22.7,10.6,16.1,9.6,18.5,5.8,23.8,28.0,13.5,14.0


In [20]:
[c for c in dftemps.columns if 'MILDURA' in c]

['MILDURA-AIRPORT_MinT_76031', 'MILDURA-AIRPORT_MaxT_76031']

In [21]:
date_features = dftemps[['Year','Month']].copy()
date_features.head(2)

,Year,Month
2009-01-01,2009,1
2009-01-02,2009,1


In [22]:
try:   # will redo these separately for the datetime features later
    dftemps.drop(columns=['Year', 'Month'], inplace=True)
except:
    pass
dftemps.head(2)

,WILLIAMTOWN-RAAF_MaxT_61078,ROCKHAMPTON_MaxT_39083,ARMIDALE_MinT_56037,MILDURA-AIRPORT_MinT_76031,CAPE-NELSON_MaxT_90184,ALBION-PARK_MaxT_68241,BUNDABERG-AERO_MaxT_39128,BROKEN-HILL-AIRPORT_MinT_47048,TOOWOOMBA-AIRPORT_MinT_41529,MOUNT-GAMBIER-AERO_MaxT_26021,...,MURRAY-BRIDGE_MaxT_24584,CANBERRA_MinT_70351,HOBART_MaxT_94029,ALBURY-AIRPORT_MinT_72160,SYDNEY-AIRPORT-AMO_MinT_66037,LAUNCESTON_MinT_91237,GLADSTONE-AIRPORT_MinT_39326,GOLD-COAST-SEAWAY_MaxT_40764,PORT-AUGUSTA_MinT_18201,DUBBO_MinT_65070
2009-01-01,34.0,37.3,6.5,13.8,15.9,34.0,34.0,18.3,18.3,17.0,...,23.8,11.1,17.5,11.3,17.7,12.6,26.3,31.0,17.5,10.8
2009-01-02,24.5,27.1,15.4,8.9,16.0,21.0,28.4,11.7,18.4,18.8,...,22.7,10.6,16.1,9.6,18.5,5.8,23.8,28.0,13.5,14.0


In [23]:
dfhols = pd.read_csv(holfile, index_col=0, parse_dates=[0])
dfhols.head(2)

,DayName,daynum,weekend,Australia,Australian Capital Territory,New South Wales,Queensland,South Australia,Tasmania,Victoria
2009-01-01,Thursday,3,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2009-01-02,Friday,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
dfhols.drop(columns='Australian Capital Territory', inplace=True) 
# am going to ignore differences between Canberra and NSW for now

In [25]:
states = ['NSW', 'QLD', 'SA', 'TAS', 'VIC']

In [26]:
hmap = {k:v for k,v in zip(dfhols.columns[-6:], [f'hol{st}' for st in ['NEM'] + states])}
hmap['daynum'] = 'weekdaynum'
print(hmap)
dfhols.rename(columns=hmap, inplace=True)
dfhols.head(2)

{'Australia': 'holNEM', 'New South Wales': 'holNSW', 'Queensland': 'holQLD', 'South Australia': 'holSA', 'Tasmania': 'holTAS', 'Victoria': 'holVIC', 'daynum': 'weekdaynum'}


,DayName,weekdaynum,weekend,holNEM,holNSW,holQLD,holSA,holTAS,holVIC
2009-01-01,Thursday,3,0,1.0,1.0,1.0,1.0,1.0,1.0
2009-01-02,Friday,4,0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
def get_workdays(inrow, instate=None):
    """for applying to rows of dataframe dfhols to determine if day is a workday or not. Set axis = 1 when applying"""
#     workday = 1
    workday = int( (inrow['weekend'] == 0) and (inrow['holNEM'] == 0) and (inrow[f'hol{instate}'] == 0))
#     workday = inrow
#     print(inrow[['weekend', 'holNEM', f'hol{instate}']])
    return workday

In [28]:
lseason_list = ['summer'] * 2 + ['shoulder'] * 3 + ['winter'] * 3 + ['shoulder'] * 3 + ['summer']
seasonmap = {k:v for k,v in zip(range(1,13), lseason_list)}
print(seasonmap)

{1: 'summer', 2: 'summer', 3: 'shoulder', 4: 'shoulder', 5: 'shoulder', 6: 'winter', 7: 'winter', 8: 'winter', 9: 'shoulder', 10: 'shoulder', 11: 'shoulder', 12: 'summer'}


In [29]:
date_features = pd.merge(date_features, pd.get_dummies(date_features.Month.map(seasonmap)), left_index=True, right_index=True)
date_features.drop(columns='Month', inplace=True)
date_features.head(2)

,Year,shoulder,summer,winter
2009-01-01,2009,0,1,0
2009-01-02,2009,0,1,0


In [30]:
for stat in states:
    date_features[f'workday{stat}'] = dfhols.apply(func=get_workdays, axis=1, args=(stat,))
date_features.head(3)

,Year,shoulder,summer,winter,workdayNSW,workdayQLD,workdaySA,workdayTAS,workdayVIC
2009-01-01,2009,0,1,0,0.0,0.0,0.0,0.0,0.0
2009-01-02,2009,0,1,0,1.0,1.0,1.0,1.0,1.0
2009-01-03,2009,0,1,0,0.0,0.0,0.0,0.0,0.0


In [31]:
date_features[(date_features.index.month == 7) & (date_features.index.year == 2018)]

,Year,shoulder,summer,winter,workdayNSW,workdayQLD,workdaySA,workdayTAS,workdayVIC
2018-07-01,2018,0,0,1,0.0,0.0,0.0,0.0,0.0
2018-07-02,2018,0,0,1,1.0,1.0,1.0,1.0,1.0
2018-07-03,2018,0,0,1,1.0,1.0,1.0,1.0,1.0
2018-07-04,2018,0,0,1,1.0,1.0,1.0,1.0,1.0
2018-07-05,2018,0,0,1,1.0,1.0,1.0,1.0,1.0
2018-07-06,2018,0,0,1,1.0,1.0,1.0,1.0,1.0
2018-07-07,2018,0,0,1,0.0,0.0,0.0,0.0,0.0
2018-07-08,2018,0,0,1,0.0,0.0,0.0,0.0,0.0
2018-07-09,2018,0,0,1,1.0,1.0,1.0,1.0,1.0
2018-07-10,2018,0,0,1,1.0,1.0,1.0,1.0,1.0


In [32]:
timdate_features =  dfdemand[['Date']].copy()
timdate_features.head(2)

,Date
SETTLEMENTDATE,
2009-01-01 00:30:00,2009-01-01
2009-01-01 01:00:00,2009-01-01


In [33]:
timdate_features['Hour_of_day'] = timdate_features.index.hour + timdate_features.index.minute / 60
timdate_features.head(2)

,Date,Hour_of_day
SETTLEMENTDATE,,
2009-01-01 00:30:00,2009-01-01,0.5
2009-01-01 01:00:00,2009-01-01,1.0


In [34]:
timdate_features = pd.merge(timdate_features, date_features, left_on='Date', right_index=True, how='left')
timdate_features.head(3)

,Date,Hour_of_day,Year,shoulder,summer,winter,workdayNSW,workdayQLD,workdaySA,workdayTAS,workdayVIC
SETTLEMENTDATE,,,,,,,,,,,
2009-01-01 00:30:00,2009-01-01,0.5,2009,0,1,0,0.0,0.0,0.0,0.0,0.0
2009-01-01 01:00:00,2009-01-01,1.0,2009,0,1,0,0.0,0.0,0.0,0.0,0.0
2009-01-01 01:30:00,2009-01-01,1.5,2009,0,1,0,0.0,0.0,0.0,0.0,0.0


In [35]:
timdate_features.index

DatetimeIndex(['2009-01-01 00:30:00', '2009-01-01 01:00:00',
               '2009-01-01 01:30:00', '2009-01-01 02:00:00',
               '2009-01-01 02:30:00', '2009-01-01 03:00:00',
               '2009-01-01 03:30:00', '2009-01-01 04:00:00',
               '2009-01-01 04:30:00', '2009-01-01 05:00:00',
               ...
               '2018-12-31 19:30:00', '2018-12-31 20:00:00',
               '2018-12-31 20:30:00', '2018-12-31 21:00:00',
               '2018-12-31 21:30:00', '2018-12-31 22:00:00',
               '2018-12-31 22:30:00', '2018-12-31 23:00:00',
               '2018-12-31 23:30:00', '2019-01-01 00:00:00'],
              dtype='datetime64[ns]', name='SETTLEMENTDATE', length=175295, freq=None)

### index is not recognised as having a frequency, need to set it

In [36]:
print(len(timdate_features))
timdate_features = timdate_features.asfreq('30T')
print(len(timdate_features))
timdate_features.tail(3)

175295
175296


,Date,Hour_of_day,Year,shoulder,summer,winter,workdayNSW,workdayQLD,workdaySA,workdayTAS,workdayVIC
SETTLEMENTDATE,,,,,,,,,,,
2018-12-31 23:00:00,2018-12-31,23.0,2018.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2018-12-31 23:30:00,2018-12-31,23.5,2018.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2019-01-01 00:00:00,2019-01-01,0.0,2019.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN


In [37]:
timdate_features.drop(index=timdate_features.index[-1], inplace=True)  ## resetting freq added a blank row at the end
timdate_features.tail(3)

,Date,Hour_of_day,Year,shoulder,summer,winter,workdayNSW,workdayQLD,workdaySA,workdayTAS,workdayVIC
SETTLEMENTDATE,,,,,,,,,,,
2018-12-31 22:30:00,2018-12-31,22.5,2018.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2018-12-31 23:00:00,2018-12-31,23.0,2018.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2018-12-31 23:30:00,2018-12-31,23.5,2018.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [38]:
timdate_features[timdate_features.isnull().any(axis=1)]

,Date,Hour_of_day,Year,shoulder,summer,winter,workdayNSW,workdayQLD,workdaySA,workdayTAS,workdayVIC
SETTLEMENTDATE,,,,,,,,,,,
2011-05-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
timdate_features.loc['2011-05-01 00:00:00','Hour_of_day'] = 0
timdate_features.fillna(method='bfill')[timdate_features.isnull().any(axis=1)]

,Date,Hour_of_day,Year,shoulder,summer,winter,workdayNSW,workdayQLD,workdaySA,workdayTAS,workdayVIC
SETTLEMENTDATE,,,,,,,,,,,
2011-05-01,2011-05-01,0.0,2011.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
timdate_features.fillna(method='bfill', inplace=True)

<font color = 'purple'><font size = 4>
    Break up BoM stations by state<br>
    ============================================================<br>

In [41]:
with open(jstempdict) as infile:
    tmpdict = json.load(infile)
print(tmpdict)

{'58161': {'Name': 'GRAFTON-AIRPORT', 'LatLong': ['-29.76', '153.03'], 'State': 'NSW'}, '66037': {'Name': 'SYDNEY-AIRPORT-AMO', 'LatLong': ['-33.95', '151.17'], 'State': 'NSW'}, '76031': {'Name': 'MILDURA-AIRPORT', 'LatLong': ['-34.24', '142.09'], 'State': 'VIC'}, '23034': {'Name': 'ADELAIDE-AIRPORT', 'LatLong': ['-34.95', '138.52'], 'State': 'SA'}, '39326': {'Name': 'GLADSTONE-AIRPORT', 'LatLong': ['-23.87', '151.22'], 'State': 'QLD'}, '86282': {'Name': 'MELBOURNE-AIRPORT', 'LatLong': ['-37.67', '144.83'], 'State': 'VIC'}, '40764': {'Name': 'GOLD-COAST-SEAWAY', 'LatLong': ['-27.94', '153.43'], 'State': 'QLD'}, '72160': {'Name': 'ALBURY-AIRPORT', 'LatLong': ['-36.07', '146.95'], 'State': 'NSW'}, '39128': {'Name': 'BUNDABERG-AERO', 'LatLong': ['-24.91', '152.32'], 'State': 'QLD'}, '61078': {'Name': 'WILLIAMTOWN-RAAF', 'LatLong': ['-32.79', '151.84'], 'State': 'NSW'}, '41529': {'Name': 'TOOWOOMBA-AIRPORT', 'LatLong': ['-27.54', '151.91'], 'State': 'QLD'}, '68241': {'Name': 'ALBION-PARK',

In [42]:
dfstations = pd.DataFrame(tmpdict).T
dfstations.head()

,LatLong,Name,State
58161,"[-29.76, 153.03]",GRAFTON-AIRPORT,NSW
66037,"[-33.95, 151.17]",SYDNEY-AIRPORT-AMO,NSW
76031,"[-34.24, 142.09]",MILDURA-AIRPORT,VIC
23034,"[-34.95, 138.52]",ADELAIDE-AIRPORT,SA
39326,"[-23.87, 151.22]",GLADSTONE-AIRPORT,QLD


In [43]:
station_map = {col: col.split('_')[-1] for col in dftemps.columns}
print(station_map)

{'WILLIAMTOWN-RAAF_MaxT_61078': '61078', 'ROCKHAMPTON_MaxT_39083': '39083', 'ARMIDALE_MinT_56037': '56037', 'MILDURA-AIRPORT_MinT_76031': '76031', 'CAPE-NELSON_MaxT_90184': '90184', 'ALBION-PARK_MaxT_68241': '68241', 'BUNDABERG-AERO_MaxT_39128': '39128', 'BROKEN-HILL-AIRPORT_MinT_47048': '47048', 'TOOWOOMBA-AIRPORT_MinT_41529': '41529', 'MOUNT-GAMBIER-AERO_MaxT_26021': '26021', 'TOWNSVILLE_MinT_32040': '32040', 'GRAFTON-AIRPORT_MaxT_58161': '58161', 'MORWELL_MaxT_85280': '85280', 'BRISBANE_MinT_40913': '40913', 'MELBOURNE-AIRPORT_MinT_86282': '86282', 'ADELAIDE-AIRPORT_MinT_23034': '23034', 'CANBERRA_MaxT_70351': '70351', 'MURRAY-BRIDGE_MinT_24584': '24584', 'HOBART_MinT_94029': '94029', 'LAUNCESTON_MaxT_91237': '91237', 'SYDNEY-AIRPORT-AMO_MaxT_66037': '66037', 'ALBURY-AIRPORT_MaxT_72160': '72160', 'GOLD-COAST-SEAWAY_MinT_40764': '40764', 'GLADSTONE-AIRPORT_MaxT_39326': '39326', 'DUBBO_MaxT_65070': '65070', 'PORT-AUGUSTA_MaxT_18201': '18201', 'WILLIAMTOWN-RAAF_MinT_61078': '61078', 'A

In [44]:
dfstations = pd.merge(pd.DataFrame(station_map, index=['statID']).T, dfstations, how='left', left_on='statID', right_index=True).reset_index()
dfstations.rename(columns={'index':'station_temp'}, inplace=True)
dfstations.head()

,station_temp,statID,LatLong,Name,State
0,WILLIAMTOWN-RAAF_MaxT_61078,61078,"[-32.79, 151.84]",WILLIAMTOWN-RAAF,NSW
1,ROCKHAMPTON_MaxT_39083,39083,"[-23.38, 150.48]",ROCKHAMPTON-AERO,QLD
2,ARMIDALE_MinT_56037,56037,"[-30.52, 151.67]",ARMIDALE,NSW
3,MILDURA-AIRPORT_MinT_76031,76031,"[-34.24, 142.09]",MILDURA-AIRPORT,VIC
4,CAPE-NELSON_MaxT_90184,90184,"[-38.43, 141.54]",CAPE-NELSON-LIGHTHOUSE,VIC


In [45]:
dfstations.State.unique()

array(['NSW', 'QLD', 'VIC', 'SA', 'TAS'], dtype=object)

In [46]:
dfstations[dfstations.Name.str.contains('MILD')]

,station_temp,statID,LatLong,Name,State
3,MILDURA-AIRPORT_MinT_76031,76031,"[-34.24, 142.09]",MILDURA-AIRPORT,VIC
29,MILDURA-AIRPORT_MaxT_76031,76031,"[-34.24, 142.09]",MILDURA-AIRPORT,VIC


In [47]:
which_stations_state = {state : dfstations.station_temp[dfstations.State == state].tolist() for state in dfstations.State.unique()}
pprint(which_stations_state)
# used for selecting which subset of temp stations belong to each state, to create dfs for each state below

{'NSW': ['WILLIAMTOWN-RAAF_MaxT_61078',
         'ARMIDALE_MinT_56037',
         'ALBION-PARK_MaxT_68241',
         'BROKEN-HILL-AIRPORT_MinT_47048',
         'GRAFTON-AIRPORT_MaxT_58161',
         'CANBERRA_MaxT_70351',
         'SYDNEY-AIRPORT-AMO_MaxT_66037',
         'ALBURY-AIRPORT_MaxT_72160',
         'DUBBO_MaxT_65070',
         'WILLIAMTOWN-RAAF_MinT_61078',
         'ALBION-PARK_MinT_68241',
         'ARMIDALE_MaxT_56037',
         'BROKEN-HILL-AIRPORT_MaxT_47048',
         'GRAFTON-AIRPORT_MinT_58161',
         'CANBERRA_MinT_70351',
         'ALBURY-AIRPORT_MinT_72160',
         'SYDNEY-AIRPORT-AMO_MinT_66037',
         'DUBBO_MinT_65070'],
 'QLD': ['ROCKHAMPTON_MaxT_39083',
         'BUNDABERG-AERO_MaxT_39128',
         'TOOWOOMBA-AIRPORT_MinT_41529',
         'TOWNSVILLE_MinT_32040',
         'BRISBANE_MinT_40913',
         'GOLD-COAST-SEAWAY_MinT_40764',
         'GLADSTONE-AIRPORT_MaxT_39326',
         'ROCKHAMPTON_MinT_39083',
         'TOOWOOMBA-AIRPORT_MaxT_41529',
 

In [48]:
## don't need 2019 data, is incomplete anyway
dftemps.drop(index=dftemps.index[dftemps.index.year == 2019], inplace=True)
dftemps.tail(3)

,WILLIAMTOWN-RAAF_MaxT_61078,ROCKHAMPTON_MaxT_39083,ARMIDALE_MinT_56037,MILDURA-AIRPORT_MinT_76031,CAPE-NELSON_MaxT_90184,ALBION-PARK_MaxT_68241,BUNDABERG-AERO_MaxT_39128,BROKEN-HILL-AIRPORT_MinT_47048,TOOWOOMBA-AIRPORT_MinT_41529,MOUNT-GAMBIER-AERO_MaxT_26021,...,MURRAY-BRIDGE_MaxT_24584,CANBERRA_MinT_70351,HOBART_MaxT_94029,ALBURY-AIRPORT_MinT_72160,SYDNEY-AIRPORT-AMO_MinT_66037,LAUNCESTON_MinT_91237,GLADSTONE-AIRPORT_MinT_39326,GOLD-COAST-SEAWAY_MaxT_40764,PORT-AUGUSTA_MinT_18201,DUBBO_MinT_65070
2018-12-29,38.4,31.1,7.4,25.9,20.9,32.0,31.2,26.6,15.8,22.5,...,28.5,16.9,25.7,27.2,20.6,16.6,22.3,29.1,24.6,21.4
2018-12-30,38.8,31.4,13.1,22.9,20.0,30.1,30.5,26.6,17.5,22.9,...,25.3,16.7,23.5,24.2,20.0,12.2,23.2,29.5,19.7,19.9
2018-12-31,36.4,30.5,12.5,21.3,21.4,30.3,30.8,27.4,17.3,26.5,...,29.5,17.7,23.2,20.5,21.1,10.3,22.1,29.7,18.9,18.8


In [49]:
dftemps.isnull().sum()

WILLIAMTOWN-RAAF_MaxT_61078        3
ROCKHAMPTON_MaxT_39083             1
ARMIDALE_MinT_56037                5
MILDURA-AIRPORT_MinT_76031         0
CAPE-NELSON_MaxT_90184             3
ALBION-PARK_MaxT_68241            47
BUNDABERG-AERO_MaxT_39128         21
BROKEN-HILL-AIRPORT_MinT_47048     7
TOOWOOMBA-AIRPORT_MinT_41529      19
MOUNT-GAMBIER-AERO_MaxT_26021      6
TOWNSVILLE_MinT_32040              3
GRAFTON-AIRPORT_MaxT_58161         6
MORWELL_MaxT_85280                 3
BRISBANE_MinT_40913               16
MELBOURNE-AIRPORT_MinT_86282       0
ADELAIDE-AIRPORT_MinT_23034        0
CANBERRA_MaxT_70351                3
MURRAY-BRIDGE_MinT_24584          14
HOBART_MinT_94029                  1
LAUNCESTON_MaxT_91237              9
SYDNEY-AIRPORT-AMO_MaxT_66037      0
ALBURY-AIRPORT_MaxT_72160         12
GOLD-COAST-SEAWAY_MinT_40764       3
GLADSTONE-AIRPORT_MaxT_39326      10
DUBBO_MaxT_65070                   7
PORT-AUGUSTA_MaxT_18201           45
WILLIAMTOWN-RAAF_MinT_61078        2
A

In [50]:
## this is just for getting Bokeh to display date time indexes in the format I want at all scales when scroll zooming
dtformats = DatetimeTickFormatter(microseconds = ['%fus'],
milliseconds = ['%3Nms', '%S.%3Ns'],
seconds = ['%Ss'],
minsec = [':%M:%S'],
minutes = [':%M', '%Mm'],
hourmin = ['%H:%M'],
hours = ['%Hh', '%H:%M'],
days = ['%d/%m', '%a%d'],
months = ['%m/%Y', '%b %Y'],
years = ['%Y'])

In [51]:
being_inspected = 'PORT-AUGUSTA_MaxT_18201'

v = figure(plot_height = 400, plot_width = 850, x_axis_type = 'datetime')
v.extra_y_ranges = {'temp' : Range1d(start=0, end=45)}
v.xaxis.formatter = dtformats
v.add_layout(LinearAxis(y_range_name='temp'), 'right')
v.line(dftemps.index, dftemps[being_inspected], color = 'black')
v.circle(dftemps.index, dftemps[being_inspected], color = 'black')
v.line(dftemps.index, dftemps[being_inspected].rolling(7,min_periods=1).mean(), color='orange')
v.line(dftemps.index, dftemps[being_inspected].rolling(7,min_periods=1,win_type='triang').mean(), color='green')
v.line(dftemps.index, dftemps[being_inspected].interpolate(), color = 'blue')
v.line(dftemps.index, dftemps.interpolate(axis=0)[being_inspected], color = 'pink')
v.vbar(dftemps.index, width=pd.to_timedelta(1, unit='D'), 
       top=dftemps[being_inspected].isnull().astype(int)*50, fill_alpha=0.3, fill_color = 'green', 
       line_alpha=0, y_range_name='temp')

show(v)

### Interpolating missing data looks to be the least-worst method, will do for entire df inplace

In [52]:
dftemps.interpolate(inplace=True)

In [53]:
dftemps.isnull().sum().sum()

0

### Final step - merge daily temperature data with half-hourly timeseries to create half-hourly df of all independent (exogenous) variables

In [54]:
testy = pd.merge(timdate_features, dftemps[which_stations_state['VIC']], how='left', left_on='Date', right_index=True)
testy.head(2)

,Date,Hour_of_day,Year,shoulder,summer,winter,workdayNSW,workdayQLD,workdaySA,workdayTAS,workdayVIC,MILDURA-AIRPORT_MinT_76031,CAPE-NELSON_MaxT_90184,MORWELL_MaxT_85280,MELBOURNE-AIRPORT_MinT_86282,CAPE-NELSON_MinT_90184,MILDURA-AIRPORT_MaxT_76031,MELBOURNE-AIRPORT_MaxT_86282,MORWELL_MinT_85280
SETTLEMENTDATE,,,,,,,,,,,,,,,,,,,
2009-01-01 00:30:00,2009-01-01,0.5,2009.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4
2009-01-01 01:00:00,2009-01-01,1.0,2009.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4


In [55]:
testy.drop(columns=[c for c in testy.columns if 'workday' in c and 'VIC' not in c], inplace=True)
testy.head()

,Date,Hour_of_day,Year,shoulder,summer,winter,workdayVIC,MILDURA-AIRPORT_MinT_76031,CAPE-NELSON_MaxT_90184,MORWELL_MaxT_85280,MELBOURNE-AIRPORT_MinT_86282,CAPE-NELSON_MinT_90184,MILDURA-AIRPORT_MaxT_76031,MELBOURNE-AIRPORT_MaxT_86282,MORWELL_MinT_85280
SETTLEMENTDATE,,,,,,,,,,,,,,,
2009-01-01 00:30:00,2009-01-01,0.5,2009.0,0.0,1.0,0.0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4
2009-01-01 01:00:00,2009-01-01,1.0,2009.0,0.0,1.0,0.0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4
2009-01-01 01:30:00,2009-01-01,1.5,2009.0,0.0,1.0,0.0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4
2009-01-01 02:00:00,2009-01-01,2.0,2009.0,0.0,1.0,0.0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4
2009-01-01 02:30:00,2009-01-01,2.5,2009.0,0.0,1.0,0.0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4


### There's a lot going on in the cell below, for each NEM region (state) it's merging the daily temperature data with the half-hourly datetime indexed data to on date, to expand out each day's temp data to repeat over the 48 half-hour intervals for that day, as shown in the test df (testy) above

In [56]:
nemregion_dfs = {}
for st in states:   # am combining two steps in one here, creating a DF from the stations in each state, merging
                    # the general datetime features
    adf = pd.merge(timdate_features, dftemps[which_stations_state[st]], how='left', left_on='Date', right_index=True)
        # then getting rid of the other workday columns for the other states, leaving only each states' workdays
    adf.drop(columns = [c for c in adf.columns if 'workday' in c and st not in c], inplace=True)
    nemregion_dfs[st] = adf

In [57]:
nemregion_dfs['VIC'].columns

Index(['Date', 'Hour_of_day', 'Year', 'shoulder', 'summer', 'winter',
       'workdayVIC', 'MILDURA-AIRPORT_MinT_76031', 'CAPE-NELSON_MaxT_90184',
       'MORWELL_MaxT_85280', 'MELBOURNE-AIRPORT_MinT_86282',
       'CAPE-NELSON_MinT_90184', 'MILDURA-AIRPORT_MaxT_76031',
       'MELBOURNE-AIRPORT_MaxT_86282', 'MORWELL_MinT_85280'],
      dtype='object')

In [58]:
nemregion_dfs['VIC'].head(3)

,Date,Hour_of_day,Year,shoulder,summer,winter,workdayVIC,MILDURA-AIRPORT_MinT_76031,CAPE-NELSON_MaxT_90184,MORWELL_MaxT_85280,MELBOURNE-AIRPORT_MinT_86282,CAPE-NELSON_MinT_90184,MILDURA-AIRPORT_MaxT_76031,MELBOURNE-AIRPORT_MaxT_86282,MORWELL_MinT_85280
SETTLEMENTDATE,,,,,,,,,,,,,,,
2009-01-01 00:30:00,2009-01-01,0.5,2009.0,0.0,1.0,0.0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4
2009-01-01 01:00:00,2009-01-01,1.0,2009.0,0.0,1.0,0.0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4
2009-01-01 01:30:00,2009-01-01,1.5,2009.0,0.0,1.0,0.0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4


In [59]:
nemregion_dfs['VIC'].index

DatetimeIndex(['2009-01-01 00:30:00', '2009-01-01 01:00:00',
               '2009-01-01 01:30:00', '2009-01-01 02:00:00',
               '2009-01-01 02:30:00', '2009-01-01 03:00:00',
               '2009-01-01 03:30:00', '2009-01-01 04:00:00',
               '2009-01-01 04:30:00', '2009-01-01 05:00:00',
               ...
               '2018-12-31 19:00:00', '2018-12-31 19:30:00',
               '2018-12-31 20:00:00', '2018-12-31 20:30:00',
               '2018-12-31 21:00:00', '2018-12-31 21:30:00',
               '2018-12-31 22:00:00', '2018-12-31 22:30:00',
               '2018-12-31 23:00:00', '2018-12-31 23:30:00'],
              dtype='datetime64[ns]', name='SETTLEMENTDATE', length=175295, freq='30T')

### Export to CSV

In [60]:
dttoday = pd.to_datetime('today').strftime("%Y%m%d")

In [61]:
for name, df in nemregion_dfs.items():            # dump to csv files, dictionary of dfs actually works fine
    df.to_csv(f'../data/ready-for-model/{dttoday}_{name}df_interpolated_features.csv')